In [ ]:
import traceback
import pandas as pd
import opensim as osim
from connections import AWS
import xml.etree.ElementTree as ET
from biomech.processing.trc import *

In [2]:
""" INITIALIZE AWS CONNECTION """
aws_connection = AWS()
aws_connection.connect()

[AWS]: Port 5433 is free.
[AWS]: Connected to RDS endpoint.


$\textbf{Inverse Kinematics: Pipeline}$

- Conservative window: BR - 0.217 seconds; then add 0.25 seconds on either side for later trimming
- Runs tool with updated template

In [3]:
from biomech.algorithms import diff_three_point

In [4]:
## OUTLINE

# connect to AWS
# load all subject info
# iterate through subjects:
    # load scaled model, IK template --> write to local dir
    # load trial TRC file --> write to local dir
    # setup & run IK tool 


# OTHER NEEDS

# ball release identification function
# trial .mot file to upload
# trial errors

In [5]:
# load a .mot file (e.g., results from an IK run)
def load_mot_file(path: str) -> pd.DataFrame:
    return pd.read_csv(path, delim_whitespace=True, skiprows=10)

# create dataframe w/ IK errors
def process_ik_errors(path: str = '_ik_marker_errors.sto',) -> tuple[pd.DataFrame, pd.DataFrame]:
    return pd.read_csv(path, sep='\t', skiprows=5, header=1)

In [6]:
# load all subject info
subject_info = aws_connection.load_subject_info()

# load all (filtered) trc files
s3_objects = aws_connection.list_s3_objects(prefix='biomechanics/subjects/')
trc_files = [obj for obj in s3_objects if obj.endswith('.trc') and '_static' not in obj and '_filtered' in obj]

# set XML dir path 
XML_DIR = 'biomechanics/xml_templates'

In [ ]:
# initialize error log, ball release times
error_log = []
ball_release_times = []

# iterate through subjects
for subject in subject_info['subject_id'].unique():
    
    try:
        # get subject info (ID -- string; throwing hand)
        subject_id = str(subject)
        throwing_hand = subject_info[subject_info['subject_id'] == subject]['throws'].values[0]

        # list all subject trials (in trc_processed) & iterate through
        subject_trials = [t for t in trc_files if f'{subject_id}' in t]
        
        if len(subject_trials) > 0:
            print(f'Processing subject {subject_id} with {len(subject_trials)} trials...')

            # load IK template & scaled model
            ik_template = aws_connection.load_xml_from_s3(f'{XML_DIR}/ik_{throwing_hand}.xml')
            scaled_model = aws_connection.load_xml_from_s3(f'biomechanics/subjects/{subject_id}/osim/{subject_id}_scaled_model.osim')

            # save model to local dir
            default_model_tree = ET.ElementTree(scaled_model)
            default_model_tree.write(f'trial_model.osim')
            
            # update IK template with scaled model, output motion file
            ik_template.find(".//model_file").text = 'trial_model.osim'
            ik_template.find(".//output_motion_file").text = f'trial.mot'

            # iterate through trials
            for trial in subject_trials:
                # load trial TRC file
                trial_trc_bytes = aws_connection.load_s3_object(trial, return_info=False)
                trial_trc_body = parse_trc_body(trial_trc_bytes, filter_markers=False, adjust_time=False)

                # save TRC file to local dir
                write_to_trc('trial.trc', trial_trc_body, throwing_hand, filter_markers=False)

                """ IK SETUP """
                # identify ball release (peak hand speed in x-direction + 5 frames)
                hand_markers = ['X7', 'Y7', 'Z7']
                trial_hand_speed = diff_three_point(trial_trc_body[hand_markers].values)
                ball_release_index = trial_hand_speed[:, 0].argmax() + 5                            # peak speed (x-direction) + 5 frames
                
                # set IK window (subtract 0.217 seconds, then ± 0.25 seconds on either side)
                ik_start_time = trial_trc_body['Time'].values[ball_release_index] - 0.217 - 0.25
                ik_end_time = min(
                    trial_trc_body['Time'].max(), 
                    trial_trc_body['Time'].values[ball_release_index] + 0.25
                )
                
                # update xml template with trial file name, time range -- > then write to local dir
                ik_template.find(".//marker_file").text = 'trial.trc'
                ik_template.find(".//time_range").text = f'{ik_start_time} {ik_end_time}'

                # write template to local dir
                ik_template_tree = ET.ElementTree(ik_template)
                ik_template_tree.write(f'trial_template_{throwing_hand}.xml')

                """ IK RUNNING """
                # initialize IK tool
                ik_tool = osim.InverseKinematicsTool(f'trial_template_{throwing_hand}.xml')
                ik_tool.run()

                # process results and errors (both as dataframes --> CSV)
                ik_results = load_mot_file('trial.mot')
                ik_errors = process_ik_errors('_ik_marker_errors.sto')

                # """ UPLOADS TO S3 """
                # set trial ID from file
                trial_id = trial.split('/')[-1].split('_filtered')[0]

                # store ball release time
                ball_release_times.append({
                    'subject_id': subject_id,
                    'study_id': trial_id,
                    'window_start': ik_start_time,
                    'ball_release_time': trial_trc_body['Time'].values[ball_release_index],
                    'window_end': ik_end_time
                })

                # upload to subject folder (non CSV)
                aws_connection.s3.upload_file(
                    'trial.mot', 
                    aws_connection.bucket_name,
                    f'biomechanics/subjects/{subject_id}/mot_processed/{trial_id}_ik.mot'
                )
                aws_connection.s3.upload_file(
                    f'trial_template_{throwing_hand}.xml', 
                    aws_connection.bucket_name,
                    f'biomechanics/subjects/{subject_id}/inverse_kinematics/{trial_id}_ik_template.xml'
                )

                # upload results to S3 (CSV)
                aws_connection.upload_to_s3(ik_results, f'biomechanics/subjects/{subject_id}/inverse_kinematics/{trial_id}_ik_results.csv')
                aws_connection.upload_to_s3(ik_errors, f'biomechanics/subjects/{subject_id}/inverse_kinematics/{trial_id}_ik_errors.csv')

            print(f'Completed processing for subject {subject_id}.')

    except Exception as e:
        print(f"Error processing subject {subject_id}: {e}")

        traceback.print_exc()

        # log error
        error_log.append({
            'subject_id': subject_id,
            'error': str(e)
        })

        continue

In [9]:
# concatenate error log
error_log_df = pd.DataFrame(error_log)

# write error log to S3 bucket
aws_connection.upload_to_s3(
    error_log_df.to_csv(index=False),
    'biomechanics/subjects/summary/error_ik_log.csv'
)

[AWS]: Uploaded object to s3://pitch-ml/biomechanics/subjects/summary/error_ik_log.csv


In [10]:
# close AWS connection
aws_connection.close()

[AWS]: Database connection closed.
[AWS]: SSH tunnel stopped.
